**Εισαγωγή Δεδομένων**


In [34]:
import pandas as pd

raw_netflix_data = pd.read_csv("netflix_titles.csv")
#netflix_data = raw_netflix_data.dropna()
netflix_data = raw_netflix_data
netflix_data.tail(3)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
7784,s7785,Movie,Zulu Man in Japan,NaN,Nasty C,NaN,"September 25, 2020",2019,TV-MA,44 min,"Documentaries, International Movies, Music & M...","In this documentary, South African rapper Nast..."
7785,s7786,TV Show,Zumbo's Just Desserts,NaN,"Adriano Zumbo, Rachel Khoo",Australia,"October 31, 2020",2019,TV-PG,1 Season,"International TV Shows, Reality TV",Dessert wizard Adriano Zumbo looks for the nex...
7786,s7787,Movie,ZZ TOP: THAT LITTLE OL' BAND FROM TEXAS,Sam Dunn,NaN,"United Kingdom, Canada, United States","March 1, 2020",2019,TV-MA,90 min,"Documentaries, Music & Musicals",This documentary delves into the mystique behi...


**Ερώτημα 1**

In [52]:
def print_most_popular_type(df):
  most_popular_type = df.type.value_counts()
  movie_amount  = most_popular_type.get('Movie')
  shows_amount = most_popular_type.get('TV Show')
  if movie_amount > shows_amount:
    print("There are more movies than TV shows (" + str(movie_amount) + " movies, " + str(shows_amount) + " TV Shows)")
  else:
    print("There are more TV Shows than movies (" + str(movie_amount) + " movies, " + str(shows_amount) + " TV Shows)")


print_most_popular_type(netflix_data)

There are more movies than TV shows (5377 movies, 2410 TV Shows)


**Ερώτημα 2**

In [53]:
def recentNetflixData(df, year):
  return netflix_data[netflix_data['release_year'] > year].reset_index()

print_most_popular_type(recentNetflixData(netflix_data, 2019))

There are more TV Shows than movies (423 movies, 476 TV Shows)


**Ερώτημα 3**

In [71]:
def print_most_popular_country(df):
  titles_by_country = df.country.value_counts()
  country_name = titles_by_country.index[0]
  movie_amount  = titles_by_country[0]
  print("Most popular country is " + str(country_name) + " with " + str(movie_amount) + " titles.")


print_most_popular_country(netflix_data)

Most popular country is United States with 2555 titles.
